
# Module 3 — AWS CLoud & Pandas

**Author:** deffip2@illinois.edu

### Objective: Using pandas library for AWS cloud data processing and bokeh for data visualization

This notebook:
1) Loads **GHCN-Daily** data from AWS S3 (anonymous) for a selected station.

2) Computes **all-time record** high/low and **1991–2020 normals** (mean) for high/low temperatures.

3) Returns a one-row `pandas.DataFrame` with columns: `['record_min_temp', 'average_min_temp', 'average_max_temp', 'record_max_temp']`.

4) Plots **record**, **normal**, and **actual** daily **high**/**low** temperatures for a chosen year.

> Data source: `https://registry.opendata.aws/noaa-ghcn/` (anonymous access)


Following  **demo.bokeh.org/weather**

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime

from bokeh.plotting import figure, show, output_notebook, output_file
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.io import save as bokeh_save

output_notebook()

Loading BokehJS ...

## 0) LOAD GHCN DATA

We load two data GHCN data: 
1. Station metadata
2. Station observation data

After loading the data, we merge those two datasets for further data processing using builtin pandas function (merge)

In [3]:
stn_ids = pd.read_fwf('http://noaa-ghcn-pds.s3.amazonaws.com/ghcnd-stations.txt', header=None, infer_nrows=1000)
stn_ids.columns = ['ID','LAT','LON','ELEV','UKN','NAME','GSN','WBAN']

periods = pd.read_fwf('http://noaa-ghcn-pds.s3.amazonaws.com/ghcnd-inventory.txt', header=None, infer_nrows=1000)
periods.columns = ['ID','LAT','LON','ELEM','TiMIN','TiMAX']

In [4]:
merged_stns = pd.merge(stn_ids,periods,how='left',left_on='ID',right_on='ID')
merged_stns = merged_stns[(merged_stns['ELEM'] == 'TMAX') & (merged_stns['TiMAX'] == 2025)]
merged_stns

,ID,LAT_x,LON_x,ELEV,UKN,NAME,GSN,WBAN,LAT_y,LON_y,ELEM,TiMIN,TiMAX
18,AE000041196,25.3330,55.517,34.0,NaN,SHARJAH INTER. AIRP,GSN,41196.0,25.3330,55.517,TMAX,1944.0,2025.0
22,AEM00041194,25.2550,55.364,10.4,NaN,DUBAI INTL,NaN,41194.0,25.2550,55.364,TMAX,1983.0,2025.0
26,AEM00041217,24.4330,54.651,26.8,NaN,ABU DHABI INTL,NaN,41217.0,24.4330,54.651,TMAX,1983.0,2025.0
30,AEM00041218,24.2620,55.609,264.9,NaN,AL AIN INTL,NaN,41218.0,24.2620,55.609,TMAX,1994.0,2025.0
54,AG000060390,36.7167,3.250,24.0,NaN,ALGER-DAR EL BEIDA,GSN,60390.0,36.7167,3.250,TMAX,1940.0,2025.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
767021,ZA000067743,-17.8170,25.817,986.0,NaN,LIVINGSTONE,GSN,67743.0,-17.8170,25.817,TMAX,1973.0,2025.0
767026,ZAM00067663,-14.4500,28.467,1207.0,NaN,KABWE/MILLIKEN,NaN,67663.0,-14.4500,28.467,TMAX,1973.0,2025.0
767038,ZI000067775,-17.9170,31.133,1480.0,NaN,HARARE (KUTSAGA),GSN,67775.0,-17.9170,31.133,TMAX,1956.0,2025.0
767079,ZI000067975,-20.0670,30.867,1095.0,NaN,MASVINGO,NaN,67975.0,-20.0670,30.867,TMAX,1951.0,2025.0


In [5]:
# Search for a desired city by changing contains()

merged_stns[merged_stns['NAME'].str.contains('DETROIT', regex=False)]

,ID,LAT_x,LON_x,ELEV,UKN,NAME,GSN,WBAN,LAT_y,LON_y,ELEM,TiMIN,TiMAX
532084,USC00212142,46.8372,-95.8375,413.0,MN,DETROIT LAKES 1 NNE,NaN,NaN,46.8372,-95.8375,TMAX,1895.0,2025.0
532107,USC00212146,46.8575,-95.6067,470.0,MN,DETROIT LAKES 12E,NaN,NaN,46.8575,-95.6067,TMAX,2009.0,2025.0
614460,USC00352292,44.7242,-122.2547,371.9,OR,DETROIT DAM,NaN,NaN,44.7242,-122.2547,TMAX,1954.0,2025.0
736574,USW00014822,42.4072,-83.0089,189.9,MI,DETROIT CITY AP,NaN,NaN,42.4072,-83.0089,TMAX,1948.0,2025.0
737676,USW00014853,42.2367,-83.5264,215.8,MI,DETROIT WILLOW RUN AP,NaN,NaN,42.2367,-83.5264,TMAX,1948.0,2025.0
763497,USW00094847,42.2311,-83.3311,192.0,MI,DETROIT METRO AP,NaN,72537.0,42.2311,-83.3311,TMAX,1958.0,2025.0


## 1) DATA LOADING FOR SELECTED CITY AND PERIOD

We use pandas parquet here for data processing efficiency. New dataframes are created for tmax and tmin

In [6]:
STATION_ID = 'USW00094847' #based on previous cell
STATION_NAME = 'DETROIT METRO AP'
YEAR = 2023
NORMALS_START = pd.Timestamp('1991-01-01')
NORMALS_END   = pd.Timestamp('2020-12-31')

S3_PATH = f's3://noaa-ghcn-pds/parquet/by_station/STATION={STATION_ID}/'
STOR = {'anon': True}


In [7]:
df = pd.read_parquet(S3_PATH, storage_options=STOR)

# We only pull good data with none Q flag and not nan data  
df_ok = df[(df['DATA_VALUE'].notna())].copy()
df_ok

,ID,DATE,DATA_VALUE,M_FLAG,Q_FLAG,S_FLAG,OBS_TIME,ELEMENT
0,USW00094847,19950709,0,None,None,X,None,ACMC
1,USW00094847,19650101,100,None,None,X,None,ACMH
2,USW00094847,19650102,100,None,None,X,None,ACMH
3,USW00094847,19650103,40,None,None,X,None,ACMH
4,USW00094847,19650104,90,None,None,X,None,ACMH
...,...,...,...,...,...,...,...,...
393740,USW00094847,19980927,1,None,None,X,None,WV03
393741,USW00094847,19990724,1,None,None,X,None,WV03
393742,USW00094847,19990928,1,None,None,X,None,WV03
393743,USW00094847,19970506,1,None,None,X,None,WV20


In [8]:
#make date the index for copied dataframe
df_ok['DATE'] = pd.to_datetime(df_ok['DATE'].apply(lambda x: datetime.strptime(x, '%Y%m%d')))
df_ok = df_ok.set_index('DATE').sort_index() 
df_ok


,ID,DATA_VALUE,M_FLAG,Q_FLAG,S_FLAG,OBS_TIME,ELEMENT
DATE,,,,,,,
1942-08-01,USW00094847,229,H,None,S,None,TAVG
1942-08-02,USW00094847,241,H,None,S,None,TAVG
1942-08-03,USW00094847,218,H,None,S,None,TAVG
1942-08-04,USW00094847,206,H,None,S,None,TAVG
1942-08-05,USW00094847,201,H,None,S,None,TAVG
...,...,...,...,...,...,...,...
2025-02-05,USW00094847,90,None,None,W,None,WDF5
2025-02-05,USW00094847,-27,None,None,W,2400,TMAX
2025-02-05,USW00094847,-71,None,None,W,2400,TMIN


In [9]:
# Creating new dataframes for each elemet of interest (here TMAX and TMIN)

df_tmax = (df_ok[df_ok['ELEMENT']=='TMAX'])

df_tmin = (df_ok[df_ok['ELEMENT']=='TMIN'])
df_tmax

,ID,DATA_VALUE,M_FLAG,Q_FLAG,S_FLAG,OBS_TIME,ELEMENT
DATE,,,,,,,
1958-12-01,USW00094847,-17,None,None,0,2400,TMAX
1958-12-02,USW00094847,17,None,None,0,2400,TMAX
1958-12-03,USW00094847,33,None,None,0,2400,TMAX
1958-12-04,USW00094847,56,None,None,0,2400,TMAX
1958-12-05,USW00094847,6,None,None,0,2400,TMAX
...,...,...,...,...,...,...,...
2025-02-01,USW00094847,-21,None,None,W,2400,TMAX
2025-02-02,USW00094847,-5,None,None,W,2400,TMAX
2025-02-03,USW00094847,61,None,None,W,2400,TMAX


In [10]:
# drop Feb 29
tmax = df_tmax[~((df_tmax.index.month==2)&(df_tmax.index.day==29))]
tmin = df_tmin[~((df_tmin.index.month==2)&(df_tmin.index.day==29))]

# annual tmax and tmin
tmax_ann = (tmax['DATA_VALUE']/10.).groupby(tmax.index.day_of_year).mean()
tmin_ann = (tmin['DATA_VALUE']/10.).groupby(tmin.index.day_of_year).mean()

x_doy = np.arange(1, 366)
dates = pd.to_datetime(pd.Series(x_doy).map(lambda d: pd.Timestamp(YEAR,1,1) + pd.Timedelta(days=d-1)))


In [11]:
tmin.head(15)

,ID,DATA_VALUE,M_FLAG,Q_FLAG,S_FLAG,OBS_TIME,ELEMENT
DATE,,,,,,,
1959-01-01,USW00094847,-6,None,None,0,2400,TMIN
1959-01-02,USW00094847,-11,None,None,0,2400,TMIN
1959-01-03,USW00094847,-67,None,None,0,2400,TMIN
1959-01-04,USW00094847,-161,None,None,0,2400,TMIN
1959-01-05,USW00094847,-206,None,None,0,2400,TMIN
1959-01-06,USW00094847,-183,None,None,0,2400,TMIN
1959-01-07,USW00094847,-100,None,None,0,2400,TMIN
1959-01-08,USW00094847,-106,None,None,0,2400,TMIN
1959-01-09,USW00094847,-128,None,None,0,2400,TMIN


## 2) DATA PROCESSING

in this section we will calculate:
1. the warmest and coldest temperature
2. Normal max and min temperature
3. Actual temperature for the selected year

In [12]:
# The warmest and coldest record
rec_hi = (tmax['DATA_VALUE'] / 10.0).groupby(tmax.index.dayofyear).max().reindex(x_doy)
rec_lo = (tmin['DATA_VALUE'] / 10.0).groupby(tmin.index.dayofyear).min().reindex(x_doy)

# Slice the data with .loc for the selected period (1991 - 2020) to generate normal values
tmax_norm_ = tmax.loc[(tmax.index >= NORMALS_START) & (tmax.index <= NORMALS_END)]
tmin_norm_ = tmin.loc[(tmin.index >= NORMALS_START) & (tmin.index <= NORMALS_END)]

nor_hi = (tmax_norm_['DATA_VALUE'] / 10.0).groupby(tmax_norm_.index.dayofyear).mean().reindex(x_doy)
nor_lo = (tmin_norm_['DATA_VALUE'] / 10.0).groupby(tmin_norm_.index.dayofyear).mean().reindex(x_doy)


In [13]:
# Slice the data with .loc for the selected actual year (e.g. 2023)
y0, y1 = pd.Timestamp(YEAR, 1, 1), pd.Timestamp(YEAR, 12, 31)

tmax_act_ = tmax.loc[(tmax.index >= y0) & (tmax.index <= y1)]
tmin_act_ = tmin.loc[(tmin.index >= y0) & (tmin.index <= y1)]

act_hi = (tmax_act_['DATA_VALUE'] / 10.0).groupby(tmax_act_.index.dayofyear).mean().reindex(x_doy)
act_lo = (tmin_act_['DATA_VALUE'] / 10.0).groupby(tmin_act_.index.dayofyear).mean().reindex(x_doy)

## 3) DATA PLOTTING WITH BOKEH

We plot the all processed data with bokeh
1. create a data source that act as a data storage for an interactive plot
2. make a an interactive plot that has hover to show information following our crusor

In [14]:
# Bokeh data source (like a pandas DataFrame for plotting purpose)

src = ColumnDataSource(dict(
    date   = dates,
    rec_hi = rec_hi.values,
    rec_lo = rec_lo.values,
    nor_hi = nor_hi.values,
    nor_lo = nor_lo.values,
    act_hi = act_hi.values,
    act_lo = act_lo.values,
))
src.column_names

['date', 'rec_hi', 'rec_lo', 'nor_hi', 'nor_lo', 'act_hi', 'act_lo']

In [15]:
temp = figure(x_axis_type='datetime', width=1000, height=520,
           title=f'{STATION_NAME} ({STATION_ID}) — Maximum and Minimum Temperature — {YEAR}')

# Bands: bounded by the highest and the lowest value
temp.varea(x='date', y1='rec_hi', y2='rec_lo', source=src, 
           fill_alpha=0.15, fill_color='dimgray', legend_label='Record Range (1958-2025)')
temp.varea(x='date', y1='nor_hi', y2='nor_lo', source=src, 
           fill_alpha=0.30, fill_color='dimgray', legend_label='Normal Range (1991–2020)')

# Actual temperature (e.g. 2023)
temp.line('date','act_hi', source=src, 
          line_width=2.5, line_color='crimson', legend_label=f'Actual High ({YEAR})')
temp.line('date','act_lo', source=src, 
          line_width=2.5, line_color='royalblue', legend_label=f'Actual Low ({YEAR})')

hover = HoverTool(
    tooltips=[
        ('Date', '@date{%b %d}'),
        ('Record High', '@rec_hi{0.0} °C'),
        ('Normal High', '@nor_hi{0.0} °C'),
        (f'Actual High {YEAR}', '@act_hi{0.0} °C'),
        ('Record Low', '@rec_lo{0.0} °C'),
        ('Normal Low', '@nor_lo{0.0} °C'),
        (f'Actual Low {YEAR}', '@act_lo{0.0} °C'),
    ],
    formatters={'@date':'datetime'},
    mode='vline'
)
temp.add_tools(hover)

temp.legend.location = 'top_left'
temp.legend.click_policy = 'hide'
temp.xaxis.axis_label = 'Date'
temp.yaxis.axis_label = 'Temperature (°C)'
temp.grid.grid_line_alpha = 0.25

show(temp)


## 4) optional: Export to HTML

The purpose is to provide the plotting result in web-based

In [37]:
output_file('ATMS523_Module3_deffip2.html', title='ATMS523_Pandas&Bokeh_deffip2')  # boleh pakai path absolut
bokeh_save(temp)

'/data/keeling/a/deffip2/ATMS_523/HW_3_Module_3/ATMS-523-Module-3-Deffip2/ATMS523_Module3_deffip2.html'